In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anime
from matplotlib.widgets import Slider
from IPython.display import Video
from sklearn.cluster import KMeans
import matplotlib.colors as mcolors

In [ ]:
## %matplotlib widget

In [ ]:
plt.rcParams["figure.figsize"] = (8,8)

# <span style="color:rgb(52, 192, 235)">1. Questão</span>

<span style="color:rgb(52, 192, 235)">Crie um IPython notebook com o nome ‘XXXX_YYYY_EA2023.ipynb’, em que XXXX 
e YYYY devem ser substituídos pelos números de aluno dos elementos constituintes do grupo. 
Neste notebook devem colocar todo o código bem como as respostas / análises.</span>

# <span style="color:rgb(102, 227, 158)">1. Resposta</span>

<span style="color:rgb(102, 227, 158)">A criação deste notebook serve como resposta a questão 1.</span>

# 

# 

------------------------------------------------------------------------ 

# 

# 

# <span style="color:rgb(52, 192, 235)">2.1. Questão</span>

<span style="color:rgb(52, 192, 235)">Prepare,  para  cada  sujeito  do  dataset,  um  dicionário  ou  array  que  contenha, 
para cada emoção, a lista de ficheiros desse sujeito e emoção. Nota: sugere-se o 
recurso às funçãos listdir() do módulo os e da função string.split() do python. </span>

# <span style="color:rgb(102, 227, 158)">2.1. Resposta</span>

In [ ]:
nome_ficheiros = os.listdir('data')

nome_ficheiros_estruturados = {}
for i in nome_ficheiros:
    actor_id = i.split('-')[-1].split('.')[0]
    emocao_id =  i.split('-')[2]

    if actor_id not in nome_ficheiros_estruturados:
        nome_ficheiros_estruturados[actor_id] = {emocao_id:[i]}
    elif emocao_id not in nome_ficheiros_estruturados[actor_id]:
        nome_ficheiros_estruturados[actor_id][emocao_id] = [i]
    else:
        nome_ficheiros_estruturados[actor_id][emocao_id].append(i)
        
nome_ficheiros_estruturados

# 

# 

------------------------------------------------------------------------------------ 

# 

# 

# <span style="color:rgb(52, 192, 235)">2.2. Questão</span>

<span style="color:rgb(52, 192, 235)">Elabore uma rotina que carregue os dados relativos a um indivíduo (ou 
todos)  e  os  devolva  num  Array  NumPy,  no  formato  [nr_de_videos  x nr_de_landmarks*coordenadas x nr_de_frames]. Note que vídeos diferentes 
têm durações (nr de frames) diferentes, pelo que deve usar o nr de frames do 
maior vídeo e usar valores NaN para completar os vídeos de duração menor.
</span>

# <span style="color:rgb(102, 227, 158)">2.2. Resposta</span>

In [ ]:
def get_ficheiros(actor='',emocao='',orderby='actor'):
    actores = {i for i in os.listdir('data') if i.split('-')[-1].split('.')[0] == actor}
    emocoes = {i for i in os.listdir('data') if i.split('-')[2] == emocao}
    if (len(actores) == 0) ^ (len(emocoes) == 0):
        nome_ficheiros = list(actores.union(emocoes))
    else:
        nome_ficheiros = list(actores.intersection(emocoes))
        
    if len(nome_ficheiros) == 0:
        if orderby == 'actor':
            nome_ficheiros = sorted(os.listdir('data'), key=lambda i: int(i.split('-')[-1].split('.')[0]))
        else:
            nome_ficheiros = sorted(os.listdir('data'), key=lambda i: int(i.split('-')[2]))

    
    return nome_ficheiros

In [ ]:
def get_maximo_frames(actor='',emocao=''):
    nome_ficheiros = get_ficheiros(actor=actor,emocao=emocao)
    nr_de_frames_maximo = 0
    i_max = 0
    for i,nome in enumerate(nome_ficheiros):
        with open(f'data/{nome}', 'r') as ficheiro:
            nr_de_frames = sum(1 for _ in ficheiro) - 1
        i_max,nr_de_frames_maximo = (i,nr_de_frames) if nr_de_frames_maximo < nr_de_frames else (i_max,nr_de_frames_maximo)
    return nr_de_frames_maximo,i_max


In [ ]:
# O numero de landmarks nos é dado no enunciado, e por isso esta aqui "hardcoded";
# As cordenadas nesse caso são x e y e tambem estao "hardcoded".
def get_dados_estruturados(actor='',emocao='',dimensao='3d',orderby='actor'):
    nr_de_landmarks = 68 
    nr_de_coordenadas = 2 
    nome_ficheiros = get_ficheiros(actor=actor,emocao=emocao,orderby=orderby)
    nr_de_videos = len(nome_ficheiros)
    nr_de_frames_maximo,i_max = get_maximo_frames(actor=actor,emocao=emocao)
    
    if dimensao == '3d':
        dados_estruturados = np.full((nr_de_videos, nr_de_landmarks*nr_de_coordenadas, nr_de_frames_maximo),np.nan)
        for i,nome in enumerate(nome_ficheiros):
            with open(f'data/{nome}', 'r') as ficheiro:
                for j,frame in enumerate(ficheiro):
                    frame = frame.split(',')
                    if j == 0:
                        index = [i for i,_ in enumerate(frame) if _.startswith('x_') or _.startswith('y_')]
                    else:
                        frame = np.array(frame)[index].astype('float')
                        dados_estruturados[i,:,j-1] = frame
        return i_max,dados_estruturados
    else:
        dados_estruturados = []
        for i,nome in enumerate(nome_ficheiros):
            actor = nome.split('-')[-1].split('.')[0]
            emocao = nome.split('-')[2]
            with open(f'data/{nome}', 'r') as ficheiro:
                for j,frame in enumerate(ficheiro):
                    frame = frame.split(',')
                    frame.extend([actor,emocao])
                    if j == 0:
                        index = [i for i,_ in enumerate(frame) if _.startswith('x_') or _.startswith('y_')]
                        index.extend([-2,-1])
                    else:
                        frame = np.array(frame)[index].astype('float')
                
                        dados_estruturados.append(frame)
        return np.array(dados_estruturados)

In [ ]:
i_max,dados_estruturados = get_dados_estruturados(actor='12',emocao='08')
# dados_estruturados = np.load('dados_estruturados.npy')

In [ ]:
dados_estruturados[i_max]

# 

# 

------------------------------------------------------------------------------------- 

# 

# 

# <span style="color:rgb(52, 192, 235)">2.3. Questão</span>

<span style="color:rgb(52, 192, 235)">Crie também uma variável Emotion com o formato [nr_de_videos x 1] onde 
guarda, para cada vídeo, um valor representante da emoção representada nesse 
vídeo (presente no nome do ficheiro CSV). 
</span>

# <span style="color:rgb(102, 227, 158)">2.3. Resposta</span>

In [ ]:
def get_emocoes(actor=''):
    nome_ficheiros = get_ficheiros(actor=actor)
    emocoes = []
    for i in nome_ficheiros:
        if actor == '':
            emocao_id = i.split('-')[2]
            emocoes.append(emocao_id)
        else:
            actor_id = i.split('-')[-1].split('.')[0]
            if actor_id == actor:
                emocao_id = i.split('-')[2]
                emocoes.append(emocao_id)
    return emocoes

In [ ]:
emocoes = get_emocoes('12')

In [ ]:
emocoes

# 

# 

---------------------------------------------------------------------------------- 

# 

# 

# <span style="color:rgb(52, 192, 235)">2.4. Questão</span>

<span style="color:rgb(52, 192, 235)">Implemente uma função de visualização das expressões, que, recebendo as 
coordenadas  dos  landmarks  relativas  a  um  frame,  gere  uma  imagem 
semelhante ao representado na figura 1.
</span>

# <span style="color:rgb(102, 227, 158)">2.4. Resposta</span>

In [ ]:
def get_frame(actor='12',emocao='01',video_id='max',frame_id=0):
    i_max,dados_estruturados = get_dados_estruturados(actor=actor,emocao=emocao)
    
    if video_id == 'max':
        video_id = i_max

    x = dados_estruturados[video_id,:68,frame_id]
    y = -dados_estruturados[video_id,68:,frame_id]
    fig, ax = plt.subplots()
    fig.set_facecolor('xkcd:midnight')
    ax.grid(False)
    ax.axis('off')
    ax.scatter(x, y, alpha=0.9, facecolors='none', linewidth=3, edgecolors='xkcd:lightblue', s=20)
    plt.show()

In [ ]:
get_frame(actor='12',emocao='08',video_id='max',frame_id=0)

# 

# 

-------------------------------------------------------------------------------------------------------

# 

# 

# <span style="color:rgb(52, 192, 235)">2.5. Questão</span>

<span style="color:rgb(52, 192, 235)">Adapte a função anterior para permitir a visualização, em sobreposição, de 
vários frames (por exemplo, todos os frames de um vídeo).
</span>

# <span style="color:rgb(102, 227, 158)">2.5. Resposta</span>

In [ ]:
def get_frames(actor='12',emocao='01',video_id='max'):
    i_max,dados_estruturados = get_dados_estruturados(actor=actor,emocao=emocao)
    
    if video_id == 'max':
        video_id = i_max
    
    frames = []
    for i,frame in enumerate(dados_estruturados[video_id].T):
        x = frame[:68]
        y = -frame[68:]
        if i == 0:
            xmin = min(x)
            xmax = max(x)
            ymin = min(y)
            ymax = max(y)
        else:
            xmin = min(x) if min(x) < xmin else xmin
            xmax = max(x) if max(x) > xmax else xmax
            ymin = min(y) if min(y) < ymin else ymin
            ymax = max(y) if max(y) > ymax else ymax
        frames.append(list(zip(x,y)))
    return frames,xmin,xmax,ymin,ymax

In [ ]:
frames,xmin,xmax,ymin,ymax = get_frames()

In [ ]:
def plot_frames(actor='12',emocao='08',video_id='max'):
    frames,xmin,xmax,ymin,ymax = get_frames()
    fig, ax = plt.subplots()
    fig.set_facecolor('xkcd:midnight')
    ax.grid(False)
    ax.axis('off')
    cmapp = plt.get_cmap('Blues')
    for i,frame in enumerate(frames[:50]):
        for p in frame:
                x = p[0]
                y = p[1]
                ax.scatter(x, y, alpha=i/len(frames[:50]), facecolors='none', linewidth=3, edgecolors=cmapp(i/len(frames[:50])), s=12)
    plt.show()

In [ ]:
plot_frames(actor='11',emocao='07',video_id='max')

# <span style="color:rgb(207, 52, 235)">2.5. BÓNUS!</span>

In [ ]:
def animate_frames(fps=100,interpolation=4,dpi=200,actor='',emocao='',filename='animated'):
    frames,xmin,xmax,ymin,ymax = get_frames(actor=actor,emocao=emocao)
    fig=plt.figure()
    fig.patch.set_facecolor('xkcd:midnight')
    p1, = plt.plot([], [], color='xkcd:lightblue', linestyle='None', marker='o',
        markerfacecolor='none',markeredgewidth=2.5,  markersize=5)


    plt.grid(False)
    plt.axis('off')

    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)


    metadata=dict(title="Movie",artist="sourabh")
    writer= anime.FFMpegWriter(fps=fps,metadata=metadata)
    with writer.saving(fig,f"{filename}.mp4",dpi):
        for i,_ in enumerate(frames):
            if i < len(frames)-1:
                intermidiate_frames_separated_cordinates = []
                current_frame = frames[i]
                next_frame = frames[i+1]

                for current_point, next_point in zip(current_frame,next_frame):
                    intermidiate = list(zip(np.linspace(current_point[0], next_point[0], interpolation),
                                            np.linspace(current_point[1], next_point[1], interpolation)))
                    intermidiate_frames_separated_cordinates.append(intermidiate)
                intermidiate_frames = list(zip(*intermidiate_frames_separated_cordinates))

                for point in intermidiate_frames:
                    point_coords = list(zip(*point))
                    x = point_coords[0]
                    y = point_coords[1]
                    p1.set_data((x,),(y,))
                    writer.grab_frame()

In [ ]:
filename_to_save = 'animation'
animate_frames(fps=100,interpolation=4,dpi=200,actor='12',emocao='08',filename=filename_to_save)
Video("animation.mp4")

#

#

---------------------------------------------------------------------------------------------

#

#

# <span style="color:rgb(52, 192, 235)">3. Questão</span>

<span style="color:rgb(52, 192, 235)">Análise e tratamento de Outliers: o objetivo será identificar e tratar outliers no dataset 
usando diferentes abordagens univariável e multivariável.  Para o efeito iremos juntar os 
frames dos vários vídeos e considerar cada frame como uma amostra, criando um dataset 
transformado com o formato [nr_total_de_frames  x nr_de_landmarks*coordenadas]. 
</span>

# <span style="color:rgb(102, 227, 158)">3. Resposta</span>

In [ ]:
dados = get_dados_estruturados(dimensao='2d',orderby='emocao')

In [ ]:
dados

# 

# 

------------------------------------------------------------------ 

# 

# 

# <span style="color:rgb(52, 192, 235)">3.1. Questão</span>

<span style="color:rgb(52, 192, 235)">Normalize os valores das coordenadas dos landmarks em cada frame em 
função do landmark de referência 27, da seguinte forma:</span>

<span style="color:rgb(52, 192, 235)">x = (x-x_ref)/x_max</span>

<span style="color:rgb(52, 192, 235)">y = (y-y_ref)/y_max</span>

<span style="color:rgb(52, 192, 235)">Considere x_max como sendo 1280 e y_max como sendo 720, as dimensões 
máximas do vídeo, em pixéis, e x_ref e y_ref como sendo os valores das 
variáveis x_27 e y_27 (coordenadas do landmark de referência 27), nesse 
mesmo frame.</span>

# <span style="color:rgb(102, 227, 158)">3.1. Resposta</span>

In [ ]:
def normalize_dados(dados):
    x = dados[:68]
    y = dados[68:]
    x_ref = x[27]
    x_max = 1280
    y_ref = y[27]
    y_max = 720
    x = (x-x_ref)/x_max
    y = (y-y_ref)/y_max
    return np.concatenate((x,y), axis=0)

In [ ]:
dados_normalizados = normalize_dados(dados[:,:-2])
dados_normalizados = np.concatenate((dados_normalizados,dados[:,-2:]),1)

#

#

-----------------

#

#

# <span style="color:rgb(52, 192, 235)">3.2. Questão</span>

<span style="color:rgb(52, 192, 235)">Elabore uma rotina que apresente simultaneamente o boxplot de cada 
emoção (8 emoções – eixo horizontal) relativo a todos os sujeitos e a um dos 
landmarks (coordenada x ou y), por exemplo, variável x_54. Sugere-se o uso 
da biblioteca matplotlib.
</span>

# <span style="color:rgb(102, 227, 158)">3.2. Resposta</span>

In [ ]:
def groupby_numpy(array, groupby=-1):
    labels = np.unique(array[:, groupby], return_index=True)
    return labels[0],np.split(array[:,:groupby], labels[1][1:])

In [ ]:
def plot_boxplot(grouped_data, target=''):

    if target == '':
        column = 0
        target = 'x_0'
    else:
        column = int(target.split('_')[-1])
        coordinate = target.split('_')[0]

        if coordinate == 'y':
            column += 68

    bg_color = 'xkcd:midnight'
    other_color = 'xkcd:lightblue'
    colors = [
              'xkcd:lightblue',
              'xkcd:carolina blue',
              'xkcd:greeny blue',
              'xkcd:blue grey',
              'xkcd:dark salmon',
              'xkcd:twilight blue',
              'xthresholdcd:royal purple',
              'xthresholdcd:light tan',
              ]
    
    data = [i[:,column] for i in grouped_data]

    fig, ax = plt.subplots(figsize=(14,6))
    fig.set_facecolor(bg_color)
    ax.set_facecolor(bg_color)
    ax.ticthreshold_params(axis='x', colors=bg_color,pad=20,size=10)
    ax.ticthreshold_params(axis='y', colors=other_color,pad=20,size=10)
    ax.spines['bottom'].set_color(bg_color)
    ax.spines['left'].set_color(bg_color)
    ax.spines['top'].set_color(bg_color)
    ax.spines['right'].set_color(bg_color)
   
    plotted = ax.boxplot(
                            data,
                            patch_artist=True,
                            widths=0.3,
                            medianprops={"color":"white",
                                        "linewidth": 0.5},
                            boxprops={"facecolor": other_color,
                                    "edgecolor": "white",
                                    "linewidth": 0.5},
                            whisthresholderprops={"color": other_color,
                                        "linewidth": 1.5},
                            capprops={"color": other_color},
                            flierprops={'marthresholder':'o',
                                        'marthresholderfacecolor':'xthresholdcd:dull red',
                                        'marthresholdersize':9,
                                        'linestyle':'none',
                                        'marthresholderedgecolor':'xthresholdcd:sthresholdy',
                                        'linewidth':1})
        
    ax.set_xticks([1,2,3,4,5,6,7,8], ['😶','😇 ','😁','😢','😡','😨','😾','😱',],fontsize=40)
    ax.set_xlabel('Emoções',fontsize=12, labelpad=20,color=other_color)
    ax.set_ylabel(f'coordenadas {target}',fontsize=12,labelpad=20,color=other_color)

    for i,v in enumerate(ax.get_xticklabels()):
        v.set_color(colors[i])

    caps = plotted['caps']
    caps = [caps[n:n+2] for n in range(0, len(caps), 2)]
    boxes = plotted['boxes']
    for i,items in enumerate(zip(caps,boxes)):
        cap, box = items
        for j in range(2):
            cap[j].set(xdata=cap[j].get_xdata() + (-0.25,0.25), linewidth=2)
        box.set(facecolor=colors[i])
    
    ax.set_xlim([0,9])
    ax.set_title('Boxplot de emoções', fontsize=15, pad=100,color=other_color)

    plt.show()

In [ ]:
dados_emocoes = np.delete(dados_normalizados, -2, 1)

In [ ]:
dados_emocoes

In [ ]:
labels,dados_por_emocoes = groupby_numpy(dados_emocoes)

In [ ]:
dados_por_emocoes[0].shape

In [ ]:
plot_boxplot(dados_por_emocoes,'y_5')

# 

# 

----------------------- 

# 

# 

# <span style="color:rgb(52, 192, 235)">3.3. Questão</span>

<span style="color:rgb(52, 192, 235)">Analise e comente a densidade de Outliers existentes no dataset, isto é, 
para  cada  variável  em  função  da  emoção.  Observe  que  a  densidade  é 
determinada recorrendo: d = (n0/nr)*100, onde n0 é o número de pontos classificados como outliers e nr é o número 
total  de  pontos.  Esta  análise  deve  ser  feita  por  variável  e  por  emoção, 
reportando depois o valor médio e desvio padrão das densidades d obtidas 
entre as diferentes variáveis para cada emoção.
</span>

# <span style="color:rgb(102, 227, 158)">3.3. Resposta</span>

In [ ]:
def detecta_outlier_iqr(array):
    q1 = np.quantile(array, 0.25)
    q3 = np.quantile(array, 0.75)
    iqr = q3-q1
    
    upper_bound = q3+(1.5*iqr)
    lower_bound = q1-(1.5*iqr)
    outliers = array[(array <= lower_bound) | (array >= upper_bound)]
    return outliers

In [ ]:
def detecta_outlier_density(array):
    outliers = detecta_outlier_iqr(array)
    densidade = (len(outliers)/len(array)) * 100
    return densidade

In [ ]:
def get_todos_outliers(dados_por_emocoes):
    stats = {}
    for i in range(8):
        stats[i] = []
        array = dados_por_emocoes[i]
        for j in array.T:
            stats[i].append(detecta_outlier_density(j))
        stats[i] = np.array(stats[i])
    return stats

In [ ]:
def get_outlier_estatisticas(dados_por_emocoes):
    stats = get_todos_outliers(dados_por_emocoes)
    outlier_stats = {c:{'mean':i.mean(),'desvio padrao':i.std()} for c,i in stats.items()}
    return outlier_stats

In [ ]:
outlier_stats = get_outlier_estatisticas(dados_por_emocoes)

In [ ]:
outlier_stats

#

#

--------------------------

#

#

# <span style="color:rgb(52, 192, 235)">3.4. Questão</span>

<span style="color:rgb(52, 192, 235)">Escreva uma rotina que receba um Array de amostras de uma variável e 
identifique os outliers usando o teste Z-Score para um k variável (parâmetro 
de entrada).
</span>

# <span style="color:rgb(102, 227, 158)">3.4. Resposta</span>

In [ ]:
def detecta_outlier_zscore(array,k=3):
    mean = array.mean()
    std = array.std() 
    outliers = []
    index = []
    for c,i in enumerate(array): 
        z = abs((i-mean)/std)
        if z > k: 
            outliers.append(i)
            index.append(c)
    return outliers

#

#

-----------------------

#

#

# <span style="color:rgb(52, 192, 235)">3.5. Questão</span>

<span style="color:rgb(52, 192, 235)">Usando o Z-score implementado assinale todos as amostras consideradas 
outliers nas variáveis do dataset. Apresente plots em que estes pontos surgem 
a vermelho, enquanto que os restantes surgem a preto. Use k=3, 3.5 e 4.
</span>

# <span style="color:rgb(102, 227, 158)">3.5. Resposta</span>

In [ ]:
def plot_boxplot_zscore(grouped_data, target='',k=3):

    if target == '':
        column = 0
        target = 'x_0'
    else:
        column = int(target.split('_')[-1])
        coordinate = target.split('_')[0]

        if coordinate == 'y':
            column += 68

    bg_color = 'xkcd:midnight'
    other_color = 'xkcd:lightblue'
    colors = [
              'xkcd:lightblue',
              'xkcd:carolina blue',
              'xkcd:greeny blue',
              'xkcd:blue grey',
              'xkcd:dark salmon',
              'xkcd:twilight blue',
              'xkcd:royal purple',
              'xkcd:light tan',
              ]

    data = [i[:,column] for i in grouped_data]

    fig, ax = plt.subplots(figsize=(14,6))
    fig.set_facecolor(bg_color)
    ax.set_facecolor(bg_color)
    ax.tick_params(axis='x', colors=bg_color,pad=20,size=10)
    ax.tick_params(axis='y', colors=other_color,pad=20,size=10)
    ax.spines['bottom'].set_color(bg_color)
    ax.spines['left'].set_color(bg_color)
    ax.spines['top'].set_color(bg_color)
    ax.spines['right'].set_color(bg_color)

    plotted = ax.boxplot(
                            data,
                            patch_artist=True,
                            widths=0.3,
                            medianprops={"color":"white",
                                        "linewidth": 0.5},
                            boxprops={"facecolor": other_color,
                                    "edgecolor": "white",
                                    "linewidth": 0.5},
                            whiskerprops={"color": other_color,
                                        "linewidth": 1.5},
                            capprops={"color": other_color},
                            flierprops={'marker':'o',
                                        'markerfacecolor':'xkcd:dull red',
                                        'markersize':9,
                                        'linestyle':'none',
                                        'markeredgecolor':'xkcd:sky',
                                        'linewidth':1}, zorder=1)

    ax.set_xticks([1,2,3,4,5,6,7,8], ['😶','😇 ','😁','😢','😡','😨','😾','😱',],fontsize=40)
    ax.set_xlabel('Emoções',fontsize=12, labelpad=20,color=other_color)
    ax.set_ylabel(f'coordenadas {target}',fontsize=12,labelpad=20,color=other_color)

    for i,v in enumerate(ax.get_xticklabels()):
        v.set_color(colors[i])

    caps = plotted['caps']
    caps = [caps[n:n+2] for n in range(0, len(caps), 2)]
    boxes = plotted['boxes']
    fliers = plotted['fliers']
    for i,items in enumerate(zip(caps,boxes)):
        cap, box = items
        box.set(facecolor=colors[i])
        x = fliers[i].get_xdata()
        y = fliers[i].get_ydata()
        outliers = detecta_outlier_zscore(y,k=k)
        if outliers:
            x = x[:len(outliers)]
            ax.scatter(x,outliers,s=75, facecolors='xkcd:fluro green', linewidth=2, edgecolors='red', zorder=1+i)
        for j in range(2):
            cap[j].set(xdata=cap[j].get_xdata() + (-0.25,0.25), linewidth=2)
        
        
    ax.set_xlim([0,9])
    ax.set_title('Boxplot de emoções', fontsize=15, pad=100,color=other_color)

    plt.show()

In [ ]:
plot_boxplot_zscore(dados_por_emocoes,target='y_5',k=4)

In [ ]:
plot_boxplot_zscore(dados_por_emocoes,target='y_5',k=3.5)

In [ ]:
plot_boxplot_zscore(dados_por_emocoes,target='y_5',k=4)

# <span style="color:rgb(207, 52, 235)">3.5. Bónus</span>

<span style="color:rgb(207, 52, 235)">Repare que correr gráficos interativos no jupyter notebook não nos permite ter uma experiência boa devido ao delay. Sendo assim, na diretoria onde este notebook se encontra existe um script chamado slider_script.py que usa os dados sample "dados_estruturados.npy" para fazer um gráfico interativo (com melhor performance).</span>

In [ ]:
# dados = np.load('dados_estruturados.npy')
# dados_emocoes = np.delete(dados, -2, 1)
# labels,dados_por_emocoes = groupby_numpy(dados_emocoes)

In [ ]:
target='y_5'
grouped_data = dados_por_emocoes
k = 3

if target == '':
    column = 0
    target = 'x_0'
else:
    column = int(target.split('_')[-1])
    coordinate = target.split('_')[0]

    if coordinate == 'y':
        column += 68

bg_color = 'xkcd:midnight'
other_color = 'xkcd:lightblue'
colors = [
            'xkcd:lightblue',
            'xkcd:carolina blue',
            'xkcd:greeny blue',
            'xkcd:blue grey',
            'xkcd:dark salmon',
            'xkcd:twilight blue',
            'xkcd:royal purple',
            'xkcd:light tan',
            ]

data = [i[:,column] for i in grouped_data]

fig, (ax,ax1) = plt.subplots(figsize=(15,7),
                             nrows=2,
                             gridspec_kw=dict(height_ratios=[10,1],
                                              hspace=1))
fig.set_facecolor(bg_color)
ax.set_facecolor(bg_color)
ax.tick_params(axis='x', colors=bg_color,pad=20,size=10)
ax.tick_params(axis='y', colors=other_color,pad=20,size=10)
ax.spines['bottom'].set_color(bg_color)
ax.spines['left'].set_color(bg_color)
ax.spines['top'].set_color(bg_color)
ax.spines['right'].set_color(bg_color)

plotted = ax.boxplot(
                        data,
                        patch_artist=True,
                        widths=0.3,
                        medianprops={"color":"white",
                                    "linewidth": 0.5},
                        boxprops={"facecolor": other_color,
                                "edgecolor": "white",
                                "linewidth": 0.5},
                        whiskerprops={"color": other_color,
                                    "linewidth": 1.5},
                        capprops={"color": other_color},
                        flierprops={'marker':'o',
                                    'markerfacecolor':'xkcd:dull red',
                                    'markersize':9,
                                    'linestyle':'none',
                                    'markeredgecolor':'xkcd:sky',
                                    'linewidth':1}, zorder=1)

ax.set_xticks([1,2,3,4,5,6,7,8], ['😶','😇 ','😁','😢','😡','😨','😾','😱',],fontsize=40)
ax.set_xlabel('Emoções',fontsize=12, labelpad=20,color=other_color)
ax.set_ylabel(f'coordenadas {target}',fontsize=12,labelpad=20,color=other_color)

for i,v in enumerate(ax.get_xticklabels()):
    v.set_color(colors[i])

caps = plotted['caps']
caps = [caps[n:n+2] for n in range(0, len(caps), 2)]
boxes = plotted['boxes']
for i,items in enumerate(zip(caps,boxes)):
    cap, box = items
    box.set(facecolor=colors[i])
    for j in range(2):
        cap[j].set(xdata=cap[j].get_xdata() + (-0.25,0.25), linewidth=2)
    
    
ax.set_xlim([0,9])
ax.set_title('Boxplot de emoções', fontsize=15, pad=100,color=other_color)

slider = Slider(ax1, "K", valmin=0, valmax=15)
slider.valtext.set_color(other_color)
slider.label.set_color(other_color)
slider.label.set_size(20)

def update(val):
    ax.clear()
    ax.set_facecolor(bg_color)
    ax.tick_params(axis='x', colors=bg_color,pad=20,size=10)
    ax.tick_params(axis='y', colors=other_color,pad=20,size=10)
    ax.spines['bottom'].set_color(bg_color)
    ax.spines['left'].set_color(bg_color)
    ax.spines['top'].set_color(bg_color)
    ax.spines['right'].set_color(bg_color)
    plotted = ax.boxplot(
                            data,
                            patch_artist=True,
                            widths=0.3,
                            medianprops={"color":"white",
                                        "linewidth": 0.5},
                            boxprops={"facecolor": other_color,
                                    "edgecolor": "white",
                                    "linewidth": 0.5},
                            whiskerprops={"color": other_color,
                                        "linewidth": 1.5},
                            capprops={"color": other_color},
                            flierprops={'marker':'o',
                                        'markerfacecolor':'xkcd:dull red',
                                        'markersize':9,
                                        'linestyle':'none',
                                        'markeredgecolor':'xkcd:sky',
                                        'linewidth':1}, zorder=1)
    
    ax.set_xticks([1,2,3,4,5,6,7,8], ['😶','😇 ','😁','😢','😡','😨','😾','😱',],fontsize=40)
    ax.set_xlabel('Emoções',fontsize=12, labelpad=20,color=other_color)
    ax.set_ylabel(f'coordenadas {target}',fontsize=12,labelpad=20,color=other_color)

    for i,v in enumerate(ax.get_xticklabels()):
        v.set_color(colors[i])

    fliers = plotted['fliers']
    for i in range(8):
        x = fliers[i].get_xdata()
        y = fliers[i].get_ydata()
        outliers = detecta_outlier_zscore(y,k=val)
        if outliers:
            x = x[:len(outliers)]
            ax.scatter(x,outliers,s=75, facecolors='xkcd:fluro green', linewidth=2, edgecolors='red', zorder=1+i)
    ax.set_xlim([0,9])
    ax.set_title('Boxplot de emoções', fontsize=15, pad=100,color=other_color)
slider.on_changed(update)
plt.show()

#

#

------------------

#

#

# <span style="color:rgb(52, 192, 235)">3.6. Questão</span>

<span style="color:rgb(52, 192, 235)">Compare e discuta os resultados obtidos em 3.2 e 3.5.
</span>

# <span style="color:rgb(102, 227, 158)">3.6. Resposta</span>

#

#

------------------

#

#

# <span style="color:rgb(52, 192, 235)">3.7. Questão</span>

<span style="color:rgb(52, 192, 235)">Elabore uma rotina que implemente o algoritmo k-means para identificar n 
(valor de entrada) clusters, recebendo agora ambas as coordenadas de um 
landmark (x e y).
</span>

# <span style="color:rgb(102, 227, 158)">3.7. Resposta</span>

In [ ]:
def normalize_dados(dados):
    x = dados[:68]
    y = dados[68:]
    x_ref = x[27]
    x_max = 1280
    y_ref = y[27]
    y_max = 720
    x = (x-x_ref)/x_max
    y = (y-y_ref)/y_max
    return np.concatenate((x,y), axis=0)

In [ ]:
def plot_kmeans_threshold(dados,p,n_clusters,threshold=50):
    x = p
    y = p + 68
    X = np.array(list(zip(dados[:,x],dados[:,y],range(len(dados[:,x])))))
    kmeans = KMeans(n_clusters=n_clusters, random_state=0,init='random', n_init="auto").fit(X[:,:2])
    kmeans_labels_ = kmeans.labels_.tolist()
    matplotlib_colors = list(mcolors.CSS4_COLORS.keys())


    for i in range(n_clusters):
        X_emotion = X[np.where(kmeans.labels_ == i)]
        mean_x = round(X_emotion[:,0].mean(),2)
        mean_y = round(X_emotion[:,1].mean(),2)
        center = (mean_x,mean_y)
        for j,point in enumerate(X_emotion[:,:2]):
            dist = round(np.linalg.norm(center - point),2)
            if dist > threshold:
                kmeans_labels_[int(X_emotion[j,2])] = 'xkcd:dull red'
            else:
                kmeans_labels_[int(X_emotion[j,2])] = matplotlib_colors[i+20]

    bg_color = 'xkcd:midnight'
    other_color = 'xkcd:lightblue'
    

    fig,ax = plt.subplots(figsize=(8,8))
    fig.set_facecolor(bg_color)
    ax.set_facecolor(bg_color)
    ax.tick_params(axis='x', colors=other_color,pad=20,size=10)
    ax.tick_params(axis='y', colors=other_color,pad=20,size=10)
    ax.spines['bottom'].set_color(bg_color)
    ax.spines['left'].set_color(bg_color)
    ax.spines['top'].set_color(bg_color)
    ax.spines['right'].set_color(bg_color)
    ax.scatter(X[:,0],X[:,1],c=kmeans_labels_)
    plt.show()

plot_kmeans_threshold(dados,30,8,threshold=100)

In [ ]:
kmeans.cluster_centers_

In [ ]:
a = np.array(['b','a','b','b','d','a','a','c','c'])
_, idx = np.unique(a, return_index=True)
print(a[np.sort(idx)])

In [ ]:
def plot_kmeans(dados,p,n,size=200):
    x = p
    y = p + 68
    X = np.array(list(zip(dados[:,x],dados[:,y],range(len(dados[:,x])))))

    # second method

    n_clusters = n**2
    cx = np.linspace(X[:,0].min(),X[:,0].max(),n).astype(int)
    cy = np.linspace(X[:,1].min(),X[:,1].max(),n).astype(int)
    centers = np.zeros((n_clusters,2))
    c = 0
    for i in cx:
        for j in cy:
            centers[c,0] = i
            centers[c,1] = j
            c += 1
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=0,init=centers, n_init=1).fit(X[:,:2])
    kmeans_labels_ = kmeans.labels_
    # _, idx = np.unique(kmeans_labels_, return_index=True)
    # kmeans_labels_unique = kmeans_labels_[np.sort(idx)]
    matplotlib_colors = list(mcolors.CSS4_COLORS.keys())
    cluster_size = []

    # for i in kmeans_labels_unique:
    #     X_emotion = X[np.where(kmeans.labels_ == i)]
    #     cluster_size.append(len(X_emotion))
    
    for i in kmeans_labels_:
        X_emotion = X[np.where(kmeans.labels_ == i)]
        if len(X_emotion) < size:
            # kmeans_labels_[int(X_emotion[i,2])] = 'xkcd:dull red'
            kmeans_labels_ = list(map(lambda x: 'xkcd:dull red' if x == i else x,kmeans_labels_))
        else:
            # kmeans_labels_[int(X_emotion[i,2])] = matplotlib_colors[i+20]
            kmeans_labels_ = list(map(lambda x: matplotlib_colors[i+60] if x == i else x,kmeans_labels_))

    bg_color = 'xkcd:midnight'
    other_color = 'xkcd:lightblue'
    colors = [
                'xkcd:lightblue',
                'xkcd:carolina blue',
                'xkcd:greeny blue',
                'xkcd:blue grey',
                'xkcd:dark salmon',
                'xkcd:twilight blue',
                'xkcd:royal purple',
                'xkcd:light tan',
                ]

    fig,ax = plt.subplots(figsize=(8,8))
    fig.set_facecolor(bg_color)
    ax.set_facecolor(bg_color)
    ax.tick_params(axis='x', colors=other_color,pad=20,size=10)
    ax.tick_params(axis='y', colors=other_color,pad=20,size=10)
    ax.spines['bottom'].set_color(bg_color)
    ax.spines['left'].set_color(bg_color)
    ax.spines['top'].set_color(bg_color)
    ax.spines['right'].set_color(bg_color)
    ax.scatter(X[:,0],X[:,1],c=kmeans_labels_,alpha=0.9)
    # ax.scatter(X[:,0],X[:,1])
    # for i,txt in zip(kmeans.cluster_centers_,cluster_size):
    #     plt.text(i[0], i[1],s=txt,ha="center",va="center")
    plt.show()
    
plot_kmeans(dados,30,5,size=5000)

In [ ]:
l

In [ ]:
plot_kmeans(dados,p=30,clusters=8)

In [ ]:
X = np.array(list(zip(dados[:,0],dados[:,68])))

In [ ]:
n = 5
clusters = n**2
spacing = int((X[:,0].max()-X[:,0].min()) / n)
cx = np.linspace(X[:,0].min(),X[:,0].max(),n).astype(int)
cy = np.linspace(X[:,1].min(),X[:,1].max(),n).astype(int)
centers = np.zeros((clusters,2))
c = 0
for i in cx:
    for j in cy:
        centers[c,0] = i
        centers[c,1] = j
        c += 1

In [ ]:
list(range(int(X[:,0].min()),int(X[:,0].max()),spacing))

In [ ]:
np.zeros((2,2))

In [ ]:
X = np.array(list(zip(dados[:,0],dados[:,68],range(len(dados[:,0])))))
kmeans = KMeans(n_clusters=8, random_state=0, n_init="auto").fit(X)

In [ ]:
k = 10
cluster_number = len(kmeans.cluster_centers_)
for i in range(cluster_number):
    X_emotion = X[np.where(kmeans.labels_ == i)]
    mean_x = round(X_emotion[:,0].mean(),2)
    mean_y = round(X_emotion[:,1].mean(),2)
    center = (mean_x,mean_y)
    outlier_index = []
    for j,point in enumerate(X_emotion[:,:2]):
        dist = round(np.linalg.norm(center - point),2)
        if dist > k:
            outlier_index.append(X_emotion[j,2])

In [ ]:
for i in X[:,:2]:
    

In [ ]:
len(outlier_index)

In [ ]:
dist

In [ ]:
data = [np.array(list(zip(i[:,0],i[:,68]))) for i in dados_por_emocoes]

In [ ]:
X = list(zip(dados_normalizados[:,0],dados_normalizados[:,68]))

In [ ]:
dados_kmeans = [list(zip(v[:68],v[68:])) for v in dados_normalizados[:,:-2]]
emocoes = dados_normalizados[:,-1]

In [ ]:
dados_kmeans[0]

In [ ]:
data = [np.array(list(zip(i[:,x],i[:,y]))) for i in dados_por_emocoes]
X = data[1]
kmeans = KMeans(n_clusters=8, random_state=0, n_init="auto").fit(X)

In [ ]:
len(kmeans.cluster_centers_)

In [ ]:
kmeans_labels = kmeans.labels_

In [ ]:
kmeans_labels[np.where(kmeans_labels == 1)]

In [ ]:
kmeans_labels = kmeans.labels_
cluster_number = len(kmeans.cluster_centers_)
for i in range(cluster_number):
    X[np.where(kmeans_labels == i)]

In [ ]:
np.concatenate((X,kmeans.labels_.reshape(-1,1)),1)

In [ ]:
kmeans.labels_[-1]

In [ ]:
groupby_numpy(np.concatenate((X,kmeans.labels_.reshape(-1,1)),1),-1)[1]